In [1]:
with open('dict_file.csv','w') as fl:
  fl.write('id_key,api_key, \n')
  for i in range(1,10000):
    line = 'ID' + str(i) + ',' + 'API_String_' + str(i) + ', \n'
    fl.write(line)

fl.close()

In [3]:
import pandas as pd

df = pd.read_csv('dict_file.csv')
print(df.head(3))
print(df.tail(3))

  id_key       api_key   
0    ID1  API_String_1   
1    ID2  API_String_2   
2    ID3  API_String_3   
      id_key          api_key   
9996  ID9997  API_String_9997   
9997  ID9998  API_String_9998   
9998  ID9999  API_String_9999   


In [ ]:
# for i in range(1, 100):
#   print(i, int(i/2500), int(i%2500))
print()

In [5]:
import requests as rq

x = 0
i = 0
def get_API(com_id):
  '''
  Books API link used to demonstrate my process. Here is sample
  https://xkcd.com//info.0.json or 
  https://xkcd.com/{commic_Id}/info.0.json  -> {commic_Id} is the variable I have to pass to fetch data 
                                                like below example  https://xkcd.com/1/info.0.json

  Note: There are only 2500 titles available in the API link.
        In order to test 10000 calls, I have to repeat same catalog of 2500 copies until 10000 calls
        from DataFrame element as input.
  '''
  str1 = 'https://xkcd.com/'
  str2 = '/info.0.json'
  #print('-->', com_id)
  j = int(int(com_id) % 2500) 
  #print('-->', j)
  if j == 0:
    j = 2500
  
  try:
    URL_LINK = str1 + str(j) + str2
    read_api = rq.get(url=URL_LINK)
    data = read_api.json()['title']
  except:
    print('#### ERROR at commic ID: ',j)
    data = '*** JSONDecodeError on commic-ID. Dummy text ****'
  #return [URL_LINK, data['title']]
  return str(data)


# for i in range(400, 1000): 
#   # if (i % 2500 == 0):
#   #   x = x + 1
#   # else:
#   #   x = int(i/2500)
#   #   j = i - (2500 * x )
#   #print(i, j, x)
#   api_data = get_API(i) #[1] 
#   print(api_data)



In [45]:
sdf = df[['id_key', 'api_key']]
print(sdf.head())
print(sdf.columns)
#print(sdf[['api_key']] )

sdf['test'] = sdf['api_key'].apply( lambda x: x.split('API_String_')[1] ) #get_API( x.split('API_String_')[1] )[1] ) 
print(sdf.head(5))
sdf['result'] = sdf['test'].apply( lambda x: get_API(x) )
print(sdf.head(5))

  id_key       api_key
0    ID1  API_String_1
1    ID2  API_String_2
2    ID3  API_String_3
3    ID4  API_String_4
4    ID5  API_String_5
Index(['id_key', 'api_key'], dtype='object')
  id_key       api_key test
0    ID1  API_String_1    1
1    ID2  API_String_2    2
2    ID3  API_String_3    3
3    ID4  API_String_4    4
4    ID5  API_String_5    5
#### ERROR so temp text 404
#### ERROR so temp text 404
#### ERROR so temp text 404
#### ERROR so temp text 404
  id_key       api_key test                result
0    ID1  API_String_1    1       Barrel - Part 1
1    ID2  API_String_2    2  Petit Trees (sketch)
2    ID3  API_String_3    3       Island (sketch)
3    ID4  API_String_4    4    Landscape (sketch)
4    ID5  API_String_5    5           Blown apart


In [47]:

import warnings
warnings.filterwarnings('ignore')

def new_col(dfx):
  dfx['api_result'] = dfx['api_key'].apply( lambda x: get_API( x.split('API_String_')[1] ) )
  return dfx


In [48]:
import numpy as np
from multiprocessing import  Pool

def df_parallel(df_temp, fn_new_col, n_cores=100):
    df_split = np.array_split(df_temp, n_cores)
    fn_pool = Pool(n_cores)
    new_df = pd.concat(fn_pool.map(fn_new_col, df_split))
    fn_pool.close()
    fn_pool.join()
    return new_df



In [50]:
rs_df = df_parallel(df, new_col)
print(rs_df)

#### ERROR so temp text 404
#### ERROR so temp text 404
#### ERROR so temp text 404
#### ERROR so temp text 404
      id_key          api_key               api_result
0        ID1     API_String_1          Barrel - Part 1
1        ID2     API_String_2     Petit Trees (sketch)
2        ID3     API_String_3          Island (sketch)
3        ID4     API_String_4       Landscape (sketch)
4        ID5     API_String_5              Blown apart
...      ...              ... ..                   ...
9994  ID9995  API_String_9995      Universal Seat Belt
9995  ID9996  API_String_9996             Mine Captcha
9996  ID9997  API_String_9997              Logic Gates
9997  ID9998  API_String_9998              Forest Walk
9998  ID9999  API_String_9999     Abandonment Function

[9999 rows x 4 columns]


In [23]:
import warnings
warnings.filterwarnings('ignore')
import threading
import pandas as pd
  
# global variable x
# x = 0
  
# def increment():
#     """
#     function to increment global variable x
#     """
#     global x
#     x += 1

#
def thread_task(lock, dfx):
    """
    task for thread
    calls increment function 100000 times.
    """
    df_ls = []
    lock.acquire()
    dfx['api_result'] = dfx['api_key'].apply( lambda x: get_API( x.split('API_String_')[1] ) )
    lock.release()
    #print(dfx.head(2))
    #df_thr = pd.concat(dfx)
    print(dfx.shape)
    return dfx
  
def main_task():
    global df_thr
    # setting global variable x as 0
    df_thr = df[:1]
  
    # creating a lock
    lock = threading.Lock()
  
    # creating threads
    t1 = threading.Thread(target=thread_task, args=(lock,df[1:100]))
    t2 = threading.Thread(target=thread_task, args=(lock,df[101:200]))
  
    # start threads
    print('t1.start')
    t1.start()
    print('t2.start')
    t2.start()
  
    # wait until threads finish their job
    t1.join()
    print('t1.join')
    t2.join()
    print('t2.join')
    #print(type(t1))
    #print(help(t1))
    #print(t1)
  
if __name__ == "__main__":
    #for i in range(10):
    main_task()
    print(df_thr)


t1.start
t2.start
(99, 4)
t1.join
(99, 4)
t2.join
<class 'threading.Thread'>
Help on Thread in module threading object:

class Thread(builtins.object)
 |  Thread(group=None, target=None, name=None, args=(), kwargs=None, *, daemon=None)
 |  
 |  A class that represents a thread of control.
 |  
 |  This class can be safely subclassed in a limited fashion. There are two ways
 |  to specify the activity: by passing a callable object to the constructor, or
 |  by overriding the run() method in a subclass.
 |  
 |  Methods defined here:
 |  
 |  __init__(self, group=None, target=None, name=None, args=(), kwargs=None, *, daemon=None)
 |      This constructor should always be called with keyword arguments. Arguments are:
 |      
 |      *group* should be None; reserved for future extension when a ThreadGroup
 |      class is implemented.
 |      
 |      *target* is the callable object to be invoked by the run()
 |      method. Defaults to None, meaning nothing is called.
 |      
 |      *n

In [21]:
print(df.shape)

(9999, 3)
